In [22]:
from imgaug import augmenters as iaa
import imageio
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import shutil

In [ ]:

# Define paths
source_root = "/home/sala/data/Soil types"  # Root folder containing multiple subfolders with images
target_folder = "/home/sala/data/soil_images"  # Folder to store renamed images

# Create target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Supported image extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

# Recursively find all image files in subfolders
image_files = []
for root, _, files in os.walk(source_root):
    for file in files:
        if os.path.splitext(file)[1].lower() in image_extensions:
            image_files.append(os.path.join(root, file))

# Rename and move images to the target folder
for i, image_path in enumerate(image_files, start=1):
    # Generate new filename (e.g., soil_0001.jpg)
    new_filename = f"soil_{i:04d}{os.path.splitext(image_path)[1]}"
    
    # Full path for the target
    target_path = os.path.join(target_folder, new_filename)
    
    # Move and rename the file
    shutil.move(image_path, target_path)
    print(f"Renamed and moved: {image_path} -> {target_path}")

print("All images have been renamed and saved in the 'soil_images' folder.")

In [ ]:

from imgaug import augmenters as iaa

# Define input folder
input_folder = "/home/sala/data/soil_images" 
augmented_folder = "/home/sala/data/augmented_soil_images"  # Folder to save augmented images

# Define augmentations
augmentations = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip with 50% probability
    iaa.Flipud(0.2),  # Vertical flip with 20% probability
    iaa.Affine(rotate=(-20, 20)),  # Random rotation between -20 and 20 degrees
    iaa.Multiply((0.8, 1.2)),  # Random brightness adjustment
    iaa.GaussianBlur(sigma=(0, 1.0))  # Apply Gaussian blur with random sigma
])

# Get list of image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]

# Augment and save images
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    
    # Check if the file exists
    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        continue
    
    # Read the image
    image = imageio.imread(image_path)
    
    # Apply augmentations
    augmented_images = augmentations(images=[image])  
    
    # Save augmented images
    for idx, aug_image in enumerate(augmented_images):
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        aug_filename = f"{base_name}_aug_{idx + 1}.jpg"
        aug_path = os.path.join(augmented_folder, aug_filename)  # Using the defined 'augmented_folder'
        # Convert RGBA to RGB if necessary
        if aug_image.shape[-1] == 4:  # Check if the image has an alpha channel
            aug_image = aug_image[:, :, :3]  # Drop the alpha channel
        
        imageio.imwrite(aug_path, aug_image)
        print(f"Saved augmented image: {aug_path}")


In [26]:

# Define paths
from sklearn.model_selection import train_test_split


synthetic_images_folder = "/home/sala/data/soil_images/"
real_images_folder = "/home/sala/data/soil_images"
output_folder = "/home/sala/data/organized_data"

# Create output directories
os.makedirs(os.path.join(output_folder, "train"), exist_ok=True)
os.makedirs(os.path.join(output_folder, "val"), exist_ok=True)
os.makedirs(os.path.join(output_folder, "test"), exist_ok=True)

# Split synthetic images
synthetic_images = [f for f in os.listdir(synthetic_images_folder) if f.endswith(('.jpg', '.png'))]
train_synthetic, temp_synthetic = train_test_split(synthetic_images, test_size=0.2, random_state=42)
val_synthetic, test_synthetic = train_test_split(temp_synthetic, test_size=0.5, random_state=42)

# Move synthetic images to respective folders
def move_images(image_list, source_folder, target_folder):
    for img in image_list:
        src_path = os.path.join(source_folder, img)
        dst_path = os.path.join(target_folder, img)
        shutil.move(src_path, dst_path)

move_images(train_synthetic, synthetic_images_folder, os.path.join(output_folder, "train"))
move_images(val_synthetic, synthetic_images_folder, os.path.join(output_folder, "val"))
move_images(test_synthetic, synthetic_images_folder, os.path.join(output_folder, "test"))

# # Reserve real-world images for testing
# real_images = [f for f in os.listdir(real_images_folder) if f.endswith(('.jpg', '.png'))]
# move_images(real_images, real_images_folder, os.path.join(output_folder, "test"))

##Synthetic Image generation

In [ ]:



def load_munsell_data(file_path):
    """
    Load Munsell Renotation Data from a CSV file.
    """
    munsell_data = pd.read_csv(file_path)
    print("Loaded Munsell Data:")
    print(munsell_data.head())
    return munsell_data


In [ ]:
def generate_synthetic_images(munsell_data, output_folder, image_size=(224, 224)):
    os.makedirs(output_folder, exist_ok=True)
    
    for index, row in munsell_data.iterrows():
        # Extract RGB values
        r, g, b = row['r'], row['g'], row['b']
        
        # Create an image with the specified color
        image = Image.new('RGB', image_size, (int(r), int(g), int(b)))
        
        # Generate a filename based on the Munsell notation
        filename = f"{row['hue']}_{row['value']}_{row['chroma']}.png"
        image_path = os.path.join(output_folder, filename)
        
        # Save the image
        image.save(image_path)
        print(f"Generated: {image_path}")



In [ ]:
munsell_data_file="/home/sala/data/equivalent_munsell.csv"
output_folder = "/home/sala/data/synthetic_images" 
munsell_data = load_munsell_data(munsell_data_file)
generate_synthetic_images(munsell_data, output_folder)
